In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 46.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 47.7 MB/s eta 0:00:0000:0100:01


In [2]:
import pandas as pd
import numpy as np

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [5]:
df = pd.read_csv('zshirah_workouts.csv')

df.head()

,Workout Timestamp,Live/On-Demand,Instructor Name,Length (minutes),Fitness Discipline,Type,Title,Class Timestamp,Total Output,Avg. Watts,Avg. Resistance,Avg. Cadence (RPM),Avg. Speed (mph),Distance (mi),Calories Burned,Avg. Heartrate,Avg. Incline,Avg. Pace (min/mi)
0,2020-12-03 18:53 (EST),On Demand,Alex Toussaint,20,Cycling,Beginner,20 min Beginner Ride,2019-12-07 10:00 (EST),100.0,84.0,33%,82.0,14.35,4.78,198.0,131.93,NaN,NaN
1,2020-12-04 16:46 (EST),On Demand,Robin Arzon,20,Cycling,Beginner,20 min Beginner Ride,2019-11-13 16:16 (EST),100.0,83.0,36%,79.0,14.69,4.89,200.0,132.59,NaN,NaN
2,2020-12-05 10:45 (EST),On Demand,Leanne Hainsby,20,Cycling,Low Impact,20 min Low Impact Ride,2019-11-21 05:22 (EST),160.0,133.0,42%,81.0,17.86,5.95,295.0,163.13,NaN,NaN
3,2020-12-06 14:40 (EST),On Demand,NaN,15,Cycling,Scenic Ride,15 min Hawaii Scenic Ride,NaN,53.0,59.0,31%,77.0,12.70,3.18,92.0,97.33,NaN,NaN
4,2020-12-06 15:35 (EST),On Demand,Selena Samuela,20,Strength,Upper Body,20 min Arms & Shoulders Strength,2020-11-25 10:00 (EST),NaN,NaN,NaN,NaN,NaN,NaN,112.0,104.31,NaN,NaN


In [7]:
df.describe()

,Length (minutes),Total Output,Avg. Watts,Avg. Cadence (RPM),Avg. Speed (mph),Distance (mi),Calories Burned,Avg. Heartrate,Avg. Incline,Avg. Pace (min/mi)
count,361.000000,207.000000,164.000000,164.000000,207.000000,207.000000,360.000000,222.000000,0.0,0.0
mean,16.584488,121.294686,114.292683,78.640244,16.849082,4.332560,160.636111,131.847297,NaN,NaN
std,11.660436,95.591948,39.051480,7.736367,2.313388,3.031311,160.092133,21.604232,NaN,NaN
min,5.000000,6.000000,42.000000,56.000000,10.000000,0.400000,3.000000,80.750000,NaN,NaN
25%,5.000000,31.500000,83.000000,73.750000,15.105000,1.355000,51.250000,113.760000,NaN,NaN
50%,15.000000,111.000000,104.500000,78.500000,17.230000,4.230000,91.000000,134.220000,NaN,NaN
75%,20.000000,186.000000,146.000000,84.000000,18.780000,6.235000,228.250000,151.657500,NaN,NaN
max,60.000000,527.000000,219.000000,96.000000,21.790000,17.720000,801.000000,168.880000,NaN,NaN


In [9]:
df.dtypes

Workout Timestamp      object
Live/On-Demand         object
Instructor Name        object
Length (minutes)        int64
Fitness Discipline     object
Type                   object
Title                  object
Class Timestamp        object
Total Output          float64
Avg. Watts            float64
Avg. Resistance        object
Avg. Cadence (RPM)    float64
Avg. Speed (mph)      float64
Distance (mi)         float64
Calories Burned       float64
Avg. Heartrate        float64
Avg. Incline          float64
Avg. Pace (min/mi)    float64
dtype: object

In [70]:
df.nunique()

Workout Timestamp     361
Live/On-Demand          2
Instructor Name        30
Length (minutes)        8
Fitness Discipline      5
Type                   22
Title                  97
Class Timestamp       303
Total Output          135
Avg. Watts             95
Avg. Resistance        30
Avg. Cadence (RPM)     34
Avg. Speed (mph)      188
Distance (mi)         169
Calories Burned       190
Avg. Heartrate        219
Avg. Incline            0
Avg. Pace (min/mi)      0
dtype: int64

In [131]:
def clean_peloton(df):
    return (df
        # [['Workout Timestamp', 'Class Timestamp']]
        .rename(
            columns=lambda c: c
                .lower()
                .replace('.','')
                .replace('-','_')
                .replace(' ','_')
                .replace('(','')
                .replace(')','')
                # .replace('/','_or_')
        )
        .rename(columns={
            'live/on_demand': 'live_or_on_demand',
            'avg_pace_min/mi': 'avg_pace_min_per_mi'
        })
        .assign(
            workout_timestamp=lambda df_:df_.workout_timestamp
                .str[:-6]
                .astype('datetime64[ns]'),
            class_timestamp=lambda df_:df_.workout_timestamp
                .fillna('')
                .astype('datetime64[ns]'),
            avg_resistance=lambda df_:df_.avg_resistance
                .str
                .strip('%')
                .astype('float'),
            week=lambda df_:df_.workout_timestamp.dt.isocalendar().week,
            month=lambda df_:df_.workout_timestamp.dt.month,
            year=lambda df_:df_.workout_timestamp.dt.year,
            date=lambda df_:df_.workout_timestamp.dt.date
        )
        .astype({
            'live_or_on_demand': 'category',
            'fitness_discipline': 'category',
        })
    )

cleaned = clean_peloton(df)
    
cleaned.head()

,workout_timestamp,live_or_on_demand,instructor_name,length_minutes,fitness_discipline,type,title,class_timestamp,total_output,avg_watts,...,avg_speed_mph,distance_mi,calories_burned,avg_heartrate,avg_incline,avg_pace_min_per_mi,week,month,year,date
0,2020-12-03 18:53:00,On Demand,Alex Toussaint,20,Cycling,Beginner,20 min Beginner Ride,2020-12-03 18:53:00,100.0,84.0,...,14.35,4.78,198.0,131.93,NaN,NaN,49,12,2020,2020-12-03
1,2020-12-04 16:46:00,On Demand,Robin Arzon,20,Cycling,Beginner,20 min Beginner Ride,2020-12-04 16:46:00,100.0,83.0,...,14.69,4.89,200.0,132.59,NaN,NaN,49,12,2020,2020-12-04
2,2020-12-05 10:45:00,On Demand,Leanne Hainsby,20,Cycling,Low Impact,20 min Low Impact Ride,2020-12-05 10:45:00,160.0,133.0,...,17.86,5.95,295.0,163.13,NaN,NaN,49,12,2020,2020-12-05
3,2020-12-06 14:40:00,On Demand,NaN,15,Cycling,Scenic Ride,15 min Hawaii Scenic Ride,2020-12-06 14:40:00,53.0,59.0,...,12.70,3.18,92.0,97.33,NaN,NaN,49,12,2020,2020-12-06
4,2020-12-06 15:35:00,On Demand,Selena Samuela,20,Strength,Upper Body,20 min Arms & Shoulders Strength,2020-12-06 15:35:00,NaN,NaN,...,NaN,NaN,112.0,104.31,NaN,NaN,49,12,2020,2020-12-06


In [124]:
cleaned

,workout_timestamp,live_or_on_demand,instructor_name,length_minutes,fitness_discipline,type,title,class_timestamp,total_output,avg_watts,...,avg_cadence_rpm,avg_speed_mph,distance_mi,calories_burned,avg_heartrate,avg_incline,avg_pace_min_per_mi,week,month,year
0,2020-12-03 18:53:00,On Demand,Alex Toussaint,20,Cycling,Beginner,20 min Beginner Ride,2020-12-03 18:53:00,100.0,84.0,...,82.0,14.35,4.78,198.0,131.93,NaN,NaN,49,12,2020
1,2020-12-04 16:46:00,On Demand,Robin Arzon,20,Cycling,Beginner,20 min Beginner Ride,2020-12-04 16:46:00,100.0,83.0,...,79.0,14.69,4.89,200.0,132.59,NaN,NaN,49,12,2020
2,2020-12-05 10:45:00,On Demand,Leanne Hainsby,20,Cycling,Low Impact,20 min Low Impact Ride,2020-12-05 10:45:00,160.0,133.0,...,81.0,17.86,5.95,295.0,163.13,NaN,NaN,49,12,2020
3,2020-12-06 14:40:00,On Demand,NaN,15,Cycling,Scenic Ride,15 min Hawaii Scenic Ride,2020-12-06 14:40:00,53.0,59.0,...,77.0,12.70,3.18,92.0,97.33,NaN,NaN,49,12,2020
4,2020-12-06 15:35:00,On Demand,Selena Samuela,20,Strength,Upper Body,20 min Arms & Shoulders Strength,2020-12-06 15:35:00,NaN,NaN,...,NaN,NaN,NaN,112.0,104.31,NaN,NaN,49,12,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2022-02-10 12:36:00,On Demand,Ben Alldis,20,Cycling,Intervals,20 min Tabata Ride,2022-02-10 12:36:00,203.0,169.0,...,77.0,19.56,6.53,290.0,NaN,NaN,NaN,6,2,2022
357,2022-02-10 12:57:00,On Demand,Ben Alldis,5,Cycling,Warm Up/Cool Down,5 min Cool Down Ride,2022-02-10 12:57:00,24.0,81.0,...,69.0,14.59,1.21,35.0,NaN,NaN,NaN,6,2,2022
358,2022-02-10 13:04:00,On Demand,Emma Lovewell,10,Strength,Core,10 min Core Strength,2022-02-10 13:04:00,NaN,NaN,...,NaN,NaN,NaN,59.0,NaN,NaN,NaN,6,2,2022
359,2022-02-11 17:30:00,On Demand,Jess Sims,15,Strength,Full Body,Flash 15,2022-02-11 17:30:00,NaN,NaN,...,NaN,NaN,NaN,89.0,NaN,NaN,NaN,6,2,2022


In [130]:
cleaned.pivot_table(
    index=['year','month'],
    columns=['fitness_discipline'],
    values='length_minutes',
    aggfunc=[np.size, 'mean'],
    fill_value=0
)

size                                     \
fitness_discipline Bike Bootcamp Cardio Cycling Strength Stretching   
year month                                                            
2020 12                        3      0      18       26          2   
2021 1                         7      1      20       13          7   
     2                         5      0      36       16          6   
     3                         1      0      11       10          3   
     4                         4      0       4        8          0   
     5                         4      2      10        2          0   
     6                         0      0       1        0          0   
     7                         2      1       7        5          0   
     8                         4      3      12        7          1   
     9                         1      0       4        4          0   
     10                        0      1       6        0          0   
     11                        4      0       3        1          0   
2022 1                         4      3      23       19          2   
     2                         4      0       9       10          1   

                            mean                                          
fitness_discipline Bike Bootcamp Cardio    Cycling   Strength Stretching  
year month                                                                
2020 12                40.000000    0.0  20.833333  15.384615   7.500000  
2021 1                 40.714286   20.0  16.250000  17.307692   8.571429  
     2                 42.000000    0.0  15.472222  18.437500   8.333333  
     3                 60.000000    0.0   8.636364  14.500000  10.000000  
     4                 33.750000    0.0  13.750000  16.875000   0.000000  
     5                 37.500000   12.5   7.500000  12.500000   0.000000  
     6                  0.000000    0.0  20.000000   0.000000   0.000000  
     7                 37.500000   20.0  13.571429  14.000000   0.000000  
     8                 41.250000   20.0  14.583333  10.000000   5.000000  
     9                 30.000000    0.0  10.000000  10.000000   0.000000  
     10                 0.000000   20.0   8.333333   0.000000   0.000000  
     11                30.000000    0.0   8.333333  15.000000   0.000000  
2022 1                 30.000000   20.0  11.739130  10.526316   5.000000  
     2                 33.750000    0.0  12.777778  11.000000   5.000000

In [133]:
cleaned

,workout_timestamp,live_or_on_demand,instructor_name,length_minutes,fitness_discipline,type,title,class_timestamp,total_output,avg_watts,...,avg_speed_mph,distance_mi,calories_burned,avg_heartrate,avg_incline,avg_pace_min_per_mi,week,month,year,date
0,2020-12-03 18:53:00,On Demand,Alex Toussaint,20,Cycling,Beginner,20 min Beginner Ride,2020-12-03 18:53:00,100.0,84.0,...,14.35,4.78,198.0,131.93,NaN,NaN,49,12,2020,2020-12-03
1,2020-12-04 16:46:00,On Demand,Robin Arzon,20,Cycling,Beginner,20 min Beginner Ride,2020-12-04 16:46:00,100.0,83.0,...,14.69,4.89,200.0,132.59,NaN,NaN,49,12,2020,2020-12-04
2,2020-12-05 10:45:00,On Demand,Leanne Hainsby,20,Cycling,Low Impact,20 min Low Impact Ride,2020-12-05 10:45:00,160.0,133.0,...,17.86,5.95,295.0,163.13,NaN,NaN,49,12,2020,2020-12-05
3,2020-12-06 14:40:00,On Demand,NaN,15,Cycling,Scenic Ride,15 min Hawaii Scenic Ride,2020-12-06 14:40:00,53.0,59.0,...,12.70,3.18,92.0,97.33,NaN,NaN,49,12,2020,2020-12-06
4,2020-12-06 15:35:00,On Demand,Selena Samuela,20,Strength,Upper Body,20 min Arms & Shoulders Strength,2020-12-06 15:35:00,NaN,NaN,...,NaN,NaN,112.0,104.31,NaN,NaN,49,12,2020,2020-12-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2022-02-10 12:36:00,On Demand,Ben Alldis,20,Cycling,Intervals,20 min Tabata Ride,2022-02-10 12:36:00,203.0,169.0,...,19.56,6.53,290.0,NaN,NaN,NaN,6,2,2022,2022-02-10
357,2022-02-10 12:57:00,On Demand,Ben Alldis,5,Cycling,Warm Up/Cool Down,5 min Cool Down Ride,2022-02-10 12:57:00,24.0,81.0,...,14.59,1.21,35.0,NaN,NaN,NaN,6,2,2022,2022-02-10
358,2022-02-10 13:04:00,On Demand,Emma Lovewell,10,Strength,Core,10 min Core Strength,2022-02-10 13:04:00,NaN,NaN,...,NaN,NaN,59.0,NaN,NaN,NaN,6,2,2022,2022-02-10
359,2022-02-11 17:30:00,On Demand,Jess Sims,15,Strength,Full Body,Flash 15,2022-02-11 17:30:00,NaN,NaN,...,NaN,NaN,89.0,NaN,NaN,NaN,6,2,2022,2022-02-11


In [132]:
cleaned.to_csv('zshirah_peloton_02112022.csv', index=False)